In [22]:
from keras.layers import Dense, Input
from keras.models import Model
from keras.applications.vgg16 import VGG16

In [32]:
import numpy as np
from keras.layers.advanced_activations import LeakyReLU
from PIL import Image
from keras.preprocessing.image import img_to_array
import os
import glob
from scipy.misc import imread,imresize
from keras.applications.vgg16 import VGG16
import skimage.color as color
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
from keras.optimizers import Adam





In [25]:
class ImageDataset:
    def __init__(self,image_weight = 224, image_height = 224):
        self.image_weight = image_weight
        self.image_height = image_height
        self.dataset_path = '../../Descargas/AVA/ava_downloader/image/'
        self.start_idx = 1
        self.end_idx = 100
        
        
        self.input_load()
        self.pred = self.pred_load()
        
    def data_input(self):
        raw_image_list = glob.glob(self.dataset.path + '*.jpg')
        raw_img_list = sorted(raw_img_list)[self.start_idx:self.end_idx]
        self.input.image = []
        self.input.hist = []
        for i, img_path in enumerate(raw_img_list):
            if i%100 == 0:
                print ("processed %d out of %d" %( i, len(raw_img_list) ))
            try:
                image = imresize(imread(img_path), (self.image_weight,self.image_height))
                image = image.img_to_array(image)
                image = np.expand_dims(image)
                image = image.img_to_array(image, axis = 0)
                image = preprocess_input(image)
                self.input.image.append(image)
                
                color_hist = lab_hist(image)
                self.input.hist.append(np.stack(np.array(color_hist)))
                
                
            except Exception as e:
                print str(e)
        
    def data_pred(self):
        pass
    #def prepare_image
    
def lab_hist(image_np):
    image_np_rgb = image_np + 0.5 # 0~1 rgb
    image_np_lab = color.rgb2lab(image_np_rgb)

    num_bin_L = 10
    num_bin_a = 10
    num_bin_b = 10

    L_max = 100
    L_min = 0
    a_max = 60
    a_min = -60
    b_max = 60
    b_min = -60
    image_np_lab = image_np_lab.reshape([224*224,3])
    H, edges = np.histogramdd(image_np_lab, bins=(num_bin_L, num_bin_a, num_bin_b), \
            range=((L_min, L_max), (a_min, a_max), (b_min, b_max)))
    return H.reshape(1000)/1000.0
        
        
        

In [38]:
class NeuralNetwork:
    def __init__(self, image_weight = 224, image_height = 224):
        self.image_size = (image_weight, image_height,3)
        self.hist_size = 100
        self.batch_size = 4
        
        optimizer = Adam(0.0002, 0.2)
        
        self.input_image = Input(shape = self.image_size)
        self.feature_NN = self.build_VGG()(self.input_image)
               
        
        self.input_hist = Input(shape = self.hist_size, name='input_hist')
        x = keras.layers.concatenate([self.feature_NN, self.input_hist])
        self.parameter_NN = self.build_NN()(x)
        
        z = Input(shape=(self.image_size))(self.input_image)
        self.output = keras.layer.concatenate([z, self.parameters_NN])

        self.combined = Model([self.input_image, self.input_hist], self.output)
        #self.combined.compile(loss = _loss_f, optimizer = optimizer)
        
        
    def build_NN(self, input_size):
        inputs = Input(shape = input_size)
        x = Dense(250, activation = 'relu')(inputs)
        x = Dense(100, activation = 'relu')(x)
        parameters = Dense(6, activation = 'linear')(x)
        model = Model(inputs=inputs, outputs=parameters)
        # model.summary
        return model
        
    def build_VGG(self, istrainable = False):
        model = VGG16(weights = 'imagenet')
        layer_name = 'fc2'
        model2 = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
        model2.trainable = istrainable
        return model2
        
class DeepFeatureNetwork:
    def __init__(self, input_size, model_func, model_path):
        self.input_tensor = tf.placeholder(tf.float32, shape=input_size)
        self.feature, self.weights = model_func(self.input_tensor, model_path)
        
    def init_model(self):
        tf.initialize_variables(self.weights).run()

    def get_feature(self, in_data):
        return sess.run(self.feature, feed_dict={self.input_tensor: in_data+0.5})
    
myNN = NeuralNetwork(224,224)

        

TypeError: 'int' object is not iterable

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 250, 250)          0         
_________________________________________________________________
dense_10 (Dense)             (None, 250, 250)          62750     
_________________________________________________________________
dense_11 (Dense)             (None, 250, 100)          25100     
_________________________________________________________________
dense_12 (Dense)             (None, 250, 6)            606       
Total params: 88,456
Trainable params: 0
Non-trainable params: 88,456
_________________________________________________________________


In [34]:
myNN = NeuralNetwork(224,224)

NameError: global name 'build_VGG' is not defined